In [34]:
# # Installing necessary dependencies
import sys
# !conda install --yes --prefix {sys.prefix} PIL
# # !{sys.executable} -m pip install ____


# !{sys.executable} -m pip install pytesseract
# !{sys.executable} -m pip install pillow
# !{sys.executable} -m pip install python-poppler
# !{sys.executable} -m pip install opencv-python
# !{sys.executable} -m pip install --upgrade pip
# !conda install -c conda-forge poppler



In [35]:
# import PyPDF2
# import re
# import win32com.client
# from win32com.client.dynamic import Dispatch
import os
from pdf2image import convert_from_path
# import pandas as pd
try:
    from PIL import Image
except ImportError:
    import Image
# import numpy as np
import pytesseract
import os

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [37]:

# FUnction to extract words from a pdf and save it in as a text file

def pdf_2_txt(path):          
    for file in os.listdir(path): # goes through the folders'
        if file.endswith('.pdf'): # finds pdfs
            p = os.path.join(path,file) # grabs path of pdf
            pdfpages = convert_from_path(p, 500) # Converting each page in the pdf into image file with 500 dpi
            image_counter = 1
            for page in pdfpages:
                filename = f"page_{image_counter}.jpg"
                # Save the image of the page in system
                page.save(filename, 'JPEG')
                image_counter+=1
                text = str(((pytesseract.image_to_string(Image.open(filename))))) # takes image and process into text
                text = text.replace('-\n', '') # some text processing
                # delete image
                os.remove(filename)

                filename = filename.replace('.jpg','') # Naming
                with open(f'{filename}.txt', 'w+') as f: # Opens a text file and writes the characters onto
                    print(text, file=f)
                    print(f'Page_{image_counter}.txt has been saved into {os.getcwd()}')
                
    return
                    
                
        


In [42]:

# path = os.path.join(os.getcwd(),'syllabus.pdf')
path = os.getcwd()
pdf_2_txt(path)
# pages = convert_from_path(path, 500) 
# for page in pages:
#     page.save('test.jpg')

Page_2.txt has been saved into C:\Users\15713\Desktop
Page_3.txt has been saved into C:\Users\15713\Desktop
Page_4.txt has been saved into C:\Users\15713\Desktop
Page_5.txt has been saved into C:\Users\15713\Desktop
Page_6.txt has been saved into C:\Users\15713\Desktop
Page_7.txt has been saved into C:\Users\15713\Desktop
Page_8.txt has been saved into C:\Users\15713\Desktop
Page_9.txt has been saved into C:\Users\15713\Desktop
Page_10.txt has been saved into C:\Users\15713\Desktop
Page_11.txt has been saved into C:\Users\15713\Desktop
